**Задача 1: Система управления библиотекой**

Создайте систему управления библиотекой используя collections:

1. Создайте namedtuple `Book` с полями: title, author, isbn, year
2. Создайте namedtuple `Reader` с полями: name, reader_id, phone
3. Используйте `defaultdict(list)` для хранения книг по жанрам
4. Используйте `deque` для очереди читателей, ожидающих популярную книгу
5. Используйте `Counter` для подсчета количества книг каждого автора
6. Используйте `OrderedDict` для хранения истории выдачи книг (читатель -> список книг)
7. Сериализуйте все данные в JSON и pickle форматы


In [ ]:
from collections import namedtuple, defaultdict, deque, Counter, OrderedDict
import json
import pickle

# 1) + 2)
Book = namedtuple('Book', ['title', 'author', 'isbn', 'year'])
Reader = namedtuple('Reader', ['name', 'reader_id', 'phone'])

# 3) Create and fill defaultdict with books
books_dict = defaultdict(list)
books_dict['scifi'].append(Book("fant1", "author1", "111", 2000))
books_dict['scifi'].append(Book("fant2", "author2", "222", 2000))
books_dict['drama'].append(Book("drama1", "author3", "333", 2000))
books_dict['drama'].append(Book("drama2", "author4", "444", 2000))
books_dict['action'].append(Book("act1", "author2", "555", 2000))
books_dict['action'].append(Book("act2", "author3", "666", 2000))

# 4) Create and fill deque with readers
readers : deque[namedtuple] = deque()
readers.append(Reader("name1", "11", "384"))
readers.append(Reader("name2", "22", "384"))
readers.append(Reader("name3", "33", "384"))
readers.append(Reader("name4", "44", "384"))

# 5) Counting authors
cnt = Counter()
for genre_books in books_dict.values():
    for book in genre_books:
        cnt[book.author] += 1

# 6)Create and fill OrderedDict
book_history : OrderedDict[namedtuple, namedtuple] = OrderedDict()
book_history[readers[0]] = books_dict["scifi"][0]
book_history[readers[1]] = books_dict["drama"][0]
book_history[readers[2]] = books_dict["action"][0]

# 7) Serialize into JSON
json_data = {
    "books_by_genre": {genre: [_._asdict() for _ in genre_books] for genre, genre_books in books_dict.items()},
    "readers_queue": [r._asdict() for r in readers],
    "author_counts": dict(cnt),
    "book_history": [
        {"reader": {"name": r.name, "reader_id": r.reader_id, "phone": r.phone},
         "book": b._asdict()}
        for r, b in book_history.items()
    ],
}

with open('library_data.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2)

# Serialize into pickle
with open('library_data.pkl', 'wb') as f:
    pickle.dump({
        "books_dict": books_dict,
        "readers": readers,
        "author_counts": cnt,
        "book_history": book_history,
    }, f)

**Задача 2: Анализатор файловой системы**

Создайте анализатор файловой системы используя os и sys:

1. Создайте namedtuple `FileInfo` с полями: name, size, extension, modified_time
2. Используйте `os.walk()` для обхода директории
3. Используйте `os.path` функции для получения информации о файлах
4. Используйте `Counter` для подсчета файлов по расширениям
5. Используйте `defaultdict(list)` для группировки файлов по размеру (маленькие < 1MB, средние 1-100MB, большие > 100MB)
6. Используйте `deque` для хранения последних 10 найденных файлов
7. Выведите статистику используя `sys.getsizeof()` для подсчета памяти
8. Сохраните результаты в JSON файл


In [ ]:
from collections import defaultdict, Counter, deque, namedtuple
import os
import sys
import json

# 0) Initialize data structures for 2), 5), 6)
extension_counter = Counter()
files_by_size = defaultdict(list)
last_10_files = deque(maxlen=10)

# 1)
FileInfo = namedtuple('FileInfo', ['name', 'size', 'extension', 'modified_time'])

# 2) Scan directory with os.walk()
for root, dirs, files in os.walk("."):
    for filename in files:
        filepath = os.path.join(root, filename)
        
        # 3) Get file information (using os.path)
        file_size = os.path.getsize(filepath)
        file_ext = os.path.splitext(filename)[1].lower()
        mod_time = os.path.getmtime(filepath)
        
        # Create FileInfo object
        file_info = FileInfo(
            name=filename,
            size=file_size,
            extension=file_ext,
            modified_time=mod_time
        )
        
        # 4) Count files by extension (using Counter lib)
        extension_counter[file_ext] += 1

        # 5) Sort files by size (using defaultdict)
        if file_size < 1024 ** 2:   # < 1MB
            files_by_size['small'].append(file_info)
        elif file_size < 100 * 1024 * 1024:   # 1-100MB
            files_by_size['medium'].append(file_info)
        else:   # > 100MB
            files_by_size['large'].append(file_info)
        
        # 6) Add to deque for last 10 files
        last_10_files.append(file_info)

# 7) Calculate memory usage (using .getsizeof())
memory_usage = {
    'extension_counter': sys.getsizeof(extension_counter),
    'files_by_size': sys.getsizeof(files_by_size),
    'last_10_files': sys.getsizeof(last_10_files),
    'total': sys.getsizeof(extension_counter) + sys.getsizeof(files_by_size) + sys.getsizeof(last_10_files)
}

# Print statistics and memory usage (using .getsizeof())
print("Files by extension:", dict(extension_counter))
print("Files by size:", {k: len(v) for k, v in files_by_size.items()})
print("Last 10 files:", [f.name for f in last_10_files])
print("Memory usage:", memory_usage)

# 8) Serialize to JSON
json_data = {
    'files_by_extension': dict(extension_counter),
    'files_by_size_category': {
        category: [f._asdict() for f in files] 
        for category, files in files_by_size.items()
    },
    'last_10_files': [f._asdict() for f in last_10_files],
    'memory_usage': memory_usage
}

with open('filesystem_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2)

**Задача 3: Система конфигурации приложения**

Создайте систему конфигурации используя ChainMap и defaultdict:

1. Создайте namedtuple `Config` с полями: key, value, section, default_value
2. Создайте несколько словарей конфигурации (default, user, environment)
3. Используйте `ChainMap` для объединения конфигураций с приоритетом
4. Используйте `defaultdict(dict)` для группировки настроек по секциям
5. Используйте `OrderedDict` для сохранения порядка загрузки конфигураций
6. Используйте `os.environ` для чтения переменных окружения
7. Сериализуйте конфигурацию в JSON и pickle форматы

In [ ]:
from collections import namedtuple, defaultdict, OrderedDict
import os

Config = namedtuple('Config', ['key', 'value', 'section', 'default_value'])

# TODO: create config dicts

# TODO: use ChainMap to merge config dicts

# TODO: defaultdist(list)

# TODO: OrderedDict

# TODO: use os.environ to read variables

# TODO: serialize into JSON/pickle